# Prompt Engineering

In the current AI world, there's probably no skill more important than **prompt engineering**. Think of prompt engineering as the practice of crafting the best input to an AI model so it can provide your users the best responses.

You can read more about Microsoft's take on prompt engineering [here](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/concepts/prompt-engineering) and more advanced techniques [here](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/concepts/advanced-prompt-engineering?pivots%253Dprogramming-language-chat-completions).

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.7.1"
#!import ../config/SettingsHelper.cs
using Microsoft.SemanticKernel;

MySettings settings = Settings.LoadFromFile();
IKernelBuilder builder = Kernel.CreateBuilder();
if (settings.Type == "azure")
    builder.AddAzureOpenAIChatCompletion(
        settings.AzureOpenAI.ChatDeployment, "gpt-35-turbo", settings.AzureOpenAI.Endpoint, settings.AzureOpenAI.ApiKey);
else
    builder.AddOpenAIChatCompletion(
        settings.OpenAI.Model, settings.OpenAI.ApiKey, settings.OpenAI.OrgId);
Kernel kernel = builder.Build();

## Start small

As with human conversations, sometimes too many words can create more confusion. Start with the most basic demand of what you want from the LLM. Only add to your prompt as you see the need to do so.

More text in your prompt means more tokens which also raises your cost.

In [ ]:
string functionDefinition = """
    Tell me how to improve my diet.
    """;
    
KernelFunction summaryFunction = kernel.CreateFunctionFromPrompt(functionDefinition);

string? completion = await summaryFunction.InvokeAsync<string>(kernel);

Console.WriteLine(completion);

# Tell AI what not to do

Something I learned a long time ago when I started managing people. You can tell them what you want them to do, but they will sometimes get a little too creative if you don't also tell them what not to do. The same applies when dealing with AI.

Another common need is to keep the AI focused on a limited number of topics. If asked about other topics, you may want the AI to ignore the question.

In [ ]:
string functionDefinition = """
    I'm overwhelmed easily so do not generate more than 3 ideas.
    As an AI life coach, you only provide me with advice about my health and wellbeing.
    If asked about anything other than individual health and wellbeing, you should respond with "I don't know". 
    
    User: Tell me how to improve my {{$input}}.
    """;
    
KernelFunction summaryFunction = kernel.CreateFunctionFromPrompt(functionDefinition);

string? completion = await summaryFunction.InvokeAsync<string>(kernel, new(){{"input", "diet"}});

Console.WriteLine($"{completion}\n");

completion = await summaryFunction.InvokeAsync<string>(kernel, new(){{"input", "electric car performance"}});

Console.WriteLine(completion);

# One-shot

One of the easiest things you can do to guide the AI is give it an example of the response you are expecting. This is known as single-shot or one-shot prompt engineering.

You might want the AI response in a particular format. Maybe you want a generated list to be comma-separated items or maybe you want a bullet list you can paste into PowerPoint or in HTML?


In [ ]:
string functionDefinition = """
    Generate a list of pet names.

    User: generate list of five dog names
    AI: Do you like: Abe | Ace | Ash | Art | Ada?

    User: generate list of five cat names
    AI: Do you like: Abe , Ace | Ash | Art | Ada?

    User: generate a list of five {{$input}} names
    AI: 
    """;
    
KernelFunction namesFunction = kernel.CreateFunctionFromPrompt(functionDefinition);

string? completion = await namesFunction.InvokeAsync<string>(kernel, new(){{"input", "cat"}});

Console.WriteLine(completion);

In the response, notice how the list is formatted with pipes between each option and how the AI is asking the user a question. 

# Few-shot

Sometimes one example isn't enough. Maybe your desired response varies dramatically based on the input.

In [ ]:
// coming soon

# Exercise

